In [31]:
import os
import boto3
from botocore.handlers import disable_signing
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from scipy.io import wavfile
import math

In [32]:
boto3.__version__

'1.15.18'

In [33]:
np.__version__

'1.18.5'

In [34]:
s3 = boto3.resource('s3')
s3.meta.client.meta.events.register('choose-signer.s3.*', disable_signing)
bucket = s3.Bucket('congo8khz-pnnn')

In [35]:
training_set = pd.read_csv('../mubin/TrainingSet_rumble_info.txt', sep = '\t')

In [36]:
training_set

,Selection,High Freq (Hz),File Offset (s),filename,duration,marginals
0,1,49.7,48860.426,nn01a_20180126_000000.wav,6.2622,NaN
1,2,47.9,48865.100,nn01a_20180126_000000.wav,3.8974,NaN
2,3,49.7,48869.829,nn01a_20180126_000000.wav,4.2672,NaN
3,4,58.1,48876.017,nn01a_20180126_000000.wav,4.0640,NaN
4,5,57.1,48880.487,nn01a_20180126_000000.wav,4.3227,NaN
...,...,...,...,...,...,...
3175,3176,44.1,86263.733,nn10b_20180728_000000.wav,2.2700,NaN
3176,3177,33.6,86264.870,nn10b_20180728_000000.wav,6.4400,NaN
3177,3178,44.1,86269.957,nn10b_20180728_000000.wav,6.3800,NaN
3178,3179,29.0,86300.628,nn10b_20180728_000000.wav,3.4700,NaN


In [37]:
def get_parent(col):
    return col.split('_')[0]

In [38]:
training_set['Parent'] = training_set['filename'].apply(get_parent)

In [39]:
training_set['Folder'] = ['Training']*len(training_set)

In [40]:
training_set['marginals'].value_counts()

marginal    945
Name: marginals, dtype: int64

In [41]:
training_set

,Selection,High Freq (Hz),File Offset (s),filename,duration,marginals,Parent,Folder
0,1,49.7,48860.426,nn01a_20180126_000000.wav,6.2622,NaN,nn01a,Training
1,2,47.9,48865.100,nn01a_20180126_000000.wav,3.8974,NaN,nn01a,Training
2,3,49.7,48869.829,nn01a_20180126_000000.wav,4.2672,NaN,nn01a,Training
3,4,58.1,48876.017,nn01a_20180126_000000.wav,4.0640,NaN,nn01a,Training
4,5,57.1,48880.487,nn01a_20180126_000000.wav,4.3227,NaN,nn01a,Training
...,...,...,...,...,...,...,...,...
3175,3176,44.1,86263.733,nn10b_20180728_000000.wav,2.2700,NaN,nn10b,Training
3176,3177,33.6,86264.870,nn10b_20180728_000000.wav,6.4400,NaN,nn10b,Training
3177,3178,44.1,86269.957,nn10b_20180728_000000.wav,6.3800,NaN,nn10b,Training
3178,3179,29.0,86300.628,nn10b_20180728_000000.wav,3.4700,NaN,nn10b,Training


In [42]:
test_set = pd.read_csv('../mubin/GeneralTest_rumble_info.txt', sep = '\t')

In [43]:
test_set

,Selection,High Freq (Hz),File Offset (s),filename,duration,marginals
0,1,36.600,29314.080,nn01d_20180127_000000.wav,4.407,NaN
1,2,31.200,32552.884,nn01d_20180127_000000.wav,3.954,NaN
2,3,39.100,40181.978,nn01d_20180127_000000.wav,6.526,NaN
3,4,40.000,22722.894,nn02e_20180906_000000.wav,5.320,NaN
4,5,30.300,22726.736,nn02e_20180906_000000.wav,3.029,NaN
...,...,...,...,...,...,...
753,754,39.400,3530.350,nn10b_20180907_000000.wav,4.359,maginal
754,755,46.600,3533.675,nn10b_20180907_000000.wav,2.139,maginal
755,756,49.300,3547.271,nn10b_20180907_000000.wav,3.510,maginal
756,757,43.624,52488.090,nn10b_20180907_000000.wav,4.492,NaN


In [44]:
test_set['Parent'] = test_set['filename'].apply(get_parent)

In [45]:
test_set['Folder'] = ['Testing']*len(test_set)

In [46]:
test_set['marginals'].unique()

array([nan, 'maginal', 'DUMMY_NoEles', 'marginal'], dtype=object)

In [47]:
test_set.drop(test_set[test_set['marginals'] == 'DUMMY_NoEles'].index, inplace = True)

In [48]:
join_set = pd.concat([training_set, test_set], ignore_index = True, join = 'outer')

In [49]:
join_set

,Selection,High Freq (Hz),File Offset (s),filename,duration,marginals,Parent,Folder
0,1,49.700,48860.426,nn01a_20180126_000000.wav,6.2622,NaN,nn01a,Training
1,2,47.900,48865.100,nn01a_20180126_000000.wav,3.8974,NaN,nn01a,Training
2,3,49.700,48869.829,nn01a_20180126_000000.wav,4.2672,NaN,nn01a,Training
3,4,58.100,48876.017,nn01a_20180126_000000.wav,4.0640,NaN,nn01a,Training
4,5,57.100,48880.487,nn01a_20180126_000000.wav,4.3227,NaN,nn01a,Training
...,...,...,...,...,...,...,...,...
3932,754,39.400,3530.350,nn10b_20180907_000000.wav,4.3590,maginal,nn10b,Testing
3933,755,46.600,3533.675,nn10b_20180907_000000.wav,2.1390,maginal,nn10b,Testing
3934,756,49.300,3547.271,nn10b_20180907_000000.wav,3.5100,maginal,nn10b,Testing
3935,757,43.624,52488.090,nn10b_20180907_000000.wav,4.4920,NaN,nn10b,Testing


In [50]:
parent_folders = join_set['Parent'].unique()
parent_folders

array(['nn01a', 'nn01b', 'nn01c', 'nn01d', 'nn01e', 'nn01f', 'nn01g',
       'nn02a', 'nn02b', 'nn02c', 'nn02d', 'nn02e', 'nn02f', 'nn02g',
       'nn03a', 'nn03b', 'nn03c', 'nn03d', 'nn03e', 'nn03f', 'nn03g',
       'nn04a', 'nn04b', 'nn04c', 'nn04d', 'nn04f', 'nn05a', 'nn05b',
       'nn05c', 'nn05d', 'nn05e', 'nn06a', 'nn06b', 'nn06c', 'nn06d',
       'nn06e', 'nn06f', 'nn07b', 'nn07c', 'nn08a', 'nn08b', 'nn08c',
       'nn09b', 'nn10a', 'nn10b', 'nn07a'], dtype=object)

In [51]:
sorted_join_set = join_set.copy()
sorted_join_set.sort_values(['filename','File Offset (s)'])
sorted_join_set.head(12)

,Selection,High Freq (Hz),File Offset (s),filename,duration,marginals,Parent,Folder
0,1,49.700,48860.4260,nn01a_20180126_000000.wav,6.2622,NaN,nn01a,Training
1,2,47.900,48865.1000,nn01a_20180126_000000.wav,3.8974,NaN,nn01a,Training
2,3,49.700,48869.8290,nn01a_20180126_000000.wav,4.2672,NaN,nn01a,Training
3,4,58.100,48876.0170,nn01a_20180126_000000.wav,4.0640,NaN,nn01a,Training
4,5,57.100,48880.4870,nn01a_20180126_000000.wav,4.3227,NaN,nn01a,Training
5,6,57.100,49017.1050,nn01a_20180126_000000.wav,3.5837,NaN,nn01a,Training
6,7,51.500,51113.7210,nn01a_20180126_000000.wav,2.9927,marginal,nn01a,Training
7,8,46.500,51117.1570,nn01a_20180126_000000.wav,3.1034,NaN,nn01a,Training
8,9,40.801,51405.6060,nn01a_20180126_000000.wav,2.4330,marginal,nn01a,Training
9,10,40.200,47942.9300,nn01a_20180203_000000.wav,1.9582,marginal,nn01a,Training


In [53]:
# Custom Append
def newAppend(ar,left,right,window,target,cuts):
    j = left
    while right-j>=window and len(target)<cuts:
        target.append(ar[j:j+window])
        j+=window


# Audio file doesnt't contain neither noise nor rumble
startingPoint = 1000

# Iterators for train and test noise set
train_noise_iterator = 1
test_noise_iterator = 1

for folder in tqdm(parent_folders):
    
    sorted_parent_df = sorted_join_set.query('Parent == @folder')
    sorted_files = sorted_parent_df['filename'].unique()
    
    for file in sorted_files:
        file_df = sorted_parent_df.query('filename == @file')
        bucket.download_file('recordings/wav/' + folder + '/' + file, './'+file)
        sr, og_audio = wavfile.read(file)
        
        rumble_tuple = []
        
        for offset, duration in zip(file_df['File Offset (s)'], file_df['duration']):
            start_sample = math.floor(sr*offset)
            end_sample = math.ceil(sr*(offset+duration))
            rumble_tuple.append((start_sample,end_sample))
        
        newAudio = []
        
        # 4 seconds cut
        cut_length = 4*sr
        total_cuts = len(file_df)
        
        i = 0
        
        while i<len(rumble_tuple)-1 and len(newAudio)<total_cuts:
            if i == 0:
                wasteSound = startingPoint*sr
                firstEnd = rumble_tuple[i][0]
                if firstEnd>wasteSound:
                    newAppend(og_audio,wasteSound+1,firstEnd,cut_length,newAudio,total_cuts)

            start = rumble_tuple[i][1]
            end = rumble_tuple[i+1][0]
            newAppend(og_audio,start+1,end,cut_length,newAudio,total_cuts)
            
            i+=1

        if len(newAudio)<total_cuts:
            endPart = rumble_tuple[len(rumble_tuple)-1][1]
            terminating_point = (len(og_audio)-1)*sr
            newAppend(og_audio,endPart,terminating_point,cut_length,newAudio,total_cuts)

        folder_to_save = file_df['Folder']
        
        newAudio_counter = 0
        print(len(newAudio),total_cuts)
        
        for save_folder in file_df['Folder']:
            if newAudio_counter < len(newAudio):
                out_audio = newAudio[newAudio_counter]

                if save_folder == 'Training':
                    wavfile.write('../mubin/Data/Training/Noise/noise_{}.wav'.format(train_noise_iterator), sr, out_audio)
                    train_noise_iterator+=1
                else:
                    wavfile.write('../mubin/Data/Testing/Noise/noise_{}.wav'.format(test_noise_iterator), sr, out_audio)
                    test_noise_iterator+=1

                newAudio_counter+=1
        os.remove('./'+file)


  0%|          | 0/46 [00:00<?, ?it/s]

9 9
3 3
1 1
47 47
3 3
1 1
9 9
8 8


  2%|▏         | 1/46 [08:32<6:24:09, 512.21s/it]

4 4
1 1
13 13
1 1
16 16
60 60


  4%|▍         | 2/46 [14:54<5:19:35, 435.80s/it]

...

 96%|█████████▌| 44/46 [3:04:22<06:16, 188.16s/it]

45 45
28 28
1 1
19 19


 98%|█████████▊| 45/46 [3:10:41<04:05, 245.37s/it]

15 15


100%|██████████| 46/46 [3:12:09<00:00, 250.65s/it]


In [54]:
training_set

,Selection,High Freq (Hz),File Offset (s),filename,duration,marginals,Parent,Folder
0,1,49.7,48860.426,nn01a_20180126_000000.wav,6.2622,NaN,nn01a,Training
1,2,47.9,48865.100,nn01a_20180126_000000.wav,3.8974,NaN,nn01a,Training
2,3,49.7,48869.829,nn01a_20180126_000000.wav,4.2672,NaN,nn01a,Training
3,4,58.1,48876.017,nn01a_20180126_000000.wav,4.0640,NaN,nn01a,Training
4,5,57.1,48880.487,nn01a_20180126_000000.wav,4.3227,NaN,nn01a,Training
...,...,...,...,...,...,...,...,...
3175,3176,44.1,86263.733,nn10b_20180728_000000.wav,2.2700,NaN,nn10b,Training
3176,3177,33.6,86264.870,nn10b_20180728_000000.wav,6.4400,NaN,nn10b,Training
3177,3178,44.1,86269.957,nn10b_20180728_000000.wav,6.3800,NaN,nn10b,Training
3178,3179,29.0,86300.628,nn10b_20180728_000000.wav,3.4700,NaN,nn10b,Training


In [55]:
len(os.listdir('../mubin/Data/Training/Noise')) == len(training_set)

True

In [56]:
test_set

,Selection,High Freq (Hz),File Offset (s),filename,duration,marginals,Parent,Folder
0,1,36.600,29314.080,nn01d_20180127_000000.wav,4.407,NaN,nn01d,Testing
1,2,31.200,32552.884,nn01d_20180127_000000.wav,3.954,NaN,nn01d,Testing
2,3,39.100,40181.978,nn01d_20180127_000000.wav,6.526,NaN,nn01d,Testing
3,4,40.000,22722.894,nn02e_20180906_000000.wav,5.320,NaN,nn02e,Testing
4,5,30.300,22726.736,nn02e_20180906_000000.wav,3.029,NaN,nn02e,Testing
...,...,...,...,...,...,...,...,...
753,754,39.400,3530.350,nn10b_20180907_000000.wav,4.359,maginal,nn10b,Testing
754,755,46.600,3533.675,nn10b_20180907_000000.wav,2.139,maginal,nn10b,Testing
755,756,49.300,3547.271,nn10b_20180907_000000.wav,3.510,maginal,nn10b,Testing
756,757,43.624,52488.090,nn10b_20180907_000000.wav,4.492,NaN,nn10b,Testing


In [58]:
len(os.listdir('../mubin/Data/Testing/Noise')) == len(test_set)

True